In [26]:
import duckdb

# Create a connection
conn = duckdb.connect(':memory:')

# Load extensions for S3 access
conn.execute("INSTALL http; LOAD http;")
conn.execute("INSTALL parquet; LOAD parquet;")

# Configure S3 connection
conn.execute("""
SET s3_endpoint='minio:9000';
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
SET s3_region='us-east-1';
SET s3_url_style='path';
""")

# Query bronze layer
bronze_df = conn.execute("SELECT * FROM read_parquet('s3://bronze/breweries/part-00000-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet') LIMIT 10").fetchdf()
print("Bronze data:")
print(bronze_df)

# Query silver layer
silver_df = conn.execute("SELECT * FROM 's3://silver/breweries' LIMIT 10").fetchdf()
print("Silver data:")
print(silver_df)

# Query gold layer
gold_df = conn.execute("SELECT * FROM 's3://gold/breweries_per_location' LIMIT 10").fetchdf()
print("Gold data:")
print(gold_df)

# Run analytics queries
analytics = conn.execute("""
SELECT country, SUM(brewery_count) as total_breweries 
FROM 's3://gold/breweries_per_location' 
GROUP BY country 
ORDER BY total_breweries DESC
""").fetchdf()
print("Analytics results:")
print(analytics)

IOException: IO Error: SSLConnection error for HTTP HEAD to 'https://minio:9000/bronze/breweries/part-00000-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet'